# Head

In [ ]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/MambaVAE")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [ ]:
%pip install datasets causal_conv1d

# Lab

In [ ]:
import VAE, importlib
importlib.reload(VAE)
from VAE import MambaVAE

In [ ]:
import torch, InitMamba, importlib
importlib.reload(InitMamba)
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

In [96]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
tokenizer.add_special_tokens({"bos_token": "<|startoftext|>"})
print("BOS token ID:", tokenizer.bos_token_id)
print("EOS token ID:", tokenizer.eos_token_id)
inputs = tokenizer(["<|startoftext|>Today is sunny. I'm happy.<|endoftext|>", "<|startoftext|>Today is sunny. I'm happy.<|endoftext|>"], return_tensors='pt') #.to('cuda')
inputs

BOS token ID: 50277
EOS token ID: 0


{'input_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])}

In [139]:
inputs = tokenizer(["<|endoftext|>", "<|endoftext|>"], return_tensors='pt', max_length=32, padding='max_length')

def add_rand_token(input_ids):
        batch_size = input_ids.shape[0]
        pos = (torch.rand(2, batch_size, device=input_ids.device) * 32).long()
        rand_token = (torch.rand(2, batch_size, device=input_ids.device) * 50276 + 1).long()
        new_input_ids = input_ids
        new_input_ids[torch.arange(batch_size),pos] = rand_token
        return new_input_ids
add_rand_token(inputs['input_ids'])

tensor([[    0,     0,     0, 21366,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
          3615,     0],
        [    0,     0,     0,     0,     0, 14497,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 45248,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0]])

In [ ]:
vae = MambaVAE().cuda()
vae.load_state_dict(torch.load('./results/result5/model.pth', weights_only=True))

In [ ]:
vae.eval()
res = vae(**inputs)
tokenizer.batch_decode(res[2].argmax(-1))

In [ ]:
vae.eval()
state = vae.encode(**inputs)
res = vae.decode(state, **inputs)
tokenizer.batch_decode(res.logits.argmax(-1))

In [ ]:
inputs2 = tokenizer(["\n"], return_tensors='pt').to('cuda')
state2 = vae.encode(**inputs2)
res = vae.decode(state, **inputs2)
tokenizer.batch_decode(res.logits.argmax(-1))

In [ ]:
inputs['input_ids'].shape

In [ ]:
model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
model2 = InitMamba.MambaModel.from_pretrained("state-spaces/mamba-130m-hf").cuda()
# model1.train()
# model2.train()

# hidden_states = model2(**inputs, layer_range = range(23))[0]
# res1 = model1(inputs_embeds = hidden_states, attention_mask = inputs['attention_mask'], layer_range = range(23, 24), output_ssm_last_states = True)
res1 = model1(**inputs, labels = inputs['input_ids'], output_ssm_last_states = True)
# ssm_last_states = res1.ssm_last_states
# print(res1[0])
print((res1.ssm_last_states).pow(2).sum()/2)

In [ ]:
res1.ssm_last_states.abs().mean()

In [ ]:
hidden_states = model2(**inputs, layer_range = range(1), inputs_ssm_states = ssm_last_states)[0]
res1 = model1(inputs_embeds = hidden_states, attention_mask = inputs['attention_mask'],
              layer_range = range(1, 24),
              output_ssm_last_states = True,
              inputs_ssm_states = ssm_last_states)
res = model1(**inputs,
             output_ssm_last_states = True,
             inputs_ssm_states = ssm_last_states)
print(res.logits-res1.logits)
print(res.logits)

# Run

In [ ]:
!python Trainer.py

In [ ]:
!python Dataset.py

In [ ]:
from google.colab import runtime
runtime.unassign()